# Data downloading

## 1.Downing allosteric site data from shsmu.edu.cn

> `data/allosteric_site_shsmu.json` 
> 
> contains data crawling form shsmu.edu.cn(home->featrue->site).

In [ ]:
# there are 1928 allosteric site in shsmu, but 1928th 3GVU can not download.

from utils.data_download import download_shsmu_as, download_rcsb, unzip

allosteric_site_index = 'data/allosteric_site_shsmu.json'

download_shsmu_as(allosteric_site_path=allosteric_site_index, outpath='data/shsmu_allosteric_site/')
unzip(origin_dir='data/shsmu_allosteric_site/', outdir='data/shsmu_allosteric_site/allosteric_site/')

!rm -r data/shsmu_allosteric_site/*.pdb.gz

download_rcsb(allosteric_site_path=allosteric_site_index, outpath='data/shsmu_allosteric_site/')

!rm -r data/shsmu_allosteric_site/3gvu.pdb.gz

unzip(origin_dir='data/shsmu_allosteric_site/', outdir='data/shsmu_allosteric_site/rscb_pdb/')

!rm -r data/shsmu_allosteric_site/*.pdb.gz

> `data/ASD_Release_201909_AS.txt` 
>
> is the description file of the allostericsite, but hard to read.

In [ ]:
from utils.data_process import transform_txt_to_csv

txt_path = 'data/ASD_Release_201909_AS.txt'
csv_path = 'data/ASD_Release_201909_AS.csv'
transform_txt_to_csv(path=txt_path, outpath=csv_path)

## 2.Download pretain data form rcsb

> `data/pretrain/list_file_protein_xray_max3A_total.txt`
>
> includes pdbids with resolution of 0.5 ~ 3A obtained by X-Ray experiment.

In [ ]:
# download logs are stored in succeed.log, timeout.log, wrang.log

from utils.data_download import download_pretarining_data, redownload_error_pretarining_data

pdblist = 'data/pretrain/list_file_protein_xray_max3A_total.txt'
output_path = 'data/pretrain/'

download_pretarining_data(list_path=pdblist, outpath=output_path)

> If there are links in timeout.log or wrang.log run redownload_error_pretarining_data() to redownload failed pdbs.

In [ ]:
timeout_path = 'data/pretrain/timeout.log'
wrang_path = 'data/pretrain/wrang.log'

redownload_error_pretarining_data(timeout_path, wrang_path, output_path)

In [ ]:
from utils.data_download import unzip

dir_path = '/mnt/g/Little-LL/pretrain/'
out_path = '/mnt/g/Little-LL/pretrain_pdb/'

unzip(dir_path, out_path)

# Data processing

In [ ]:
s = "ATOM      1  N   PHE A   3     127.591  57.948 -15.781  0.50 28.16           N  "
print(s[0:4], s[21], s[17:20], len(s[22:26]), len(s[22:26].strip()))

In [1]:
from utils.data_process import build_tokenizer_dataset, tokenizer_json_to_txt

pdb_paths = '/mnt/g/Little-LL/pretrain_pdb/'
output_path ='/mnt/g/Little-LL/pretrain_tokenizer/'

build_tokenizer_dataset(path=pdb_paths, outpath=output_path)
tokenizer_json_to_txt(output_path)

  1%|          | 82/10000 [00:00<01:41, 97.85it/s] 

pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'
pdbid: 155C	chain: A	error: 'UNK'


  9%|▊         | 863/10000 [00:21<06:08, 24.77it/s]

pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	chain: C	error: 'UNK'
pdbid: 1ATI	ch

 12%|█▏        | 1241/10000 [00:39<08:32, 17.08it/s]

...

 68%|██████▊   | 5917/8729 [19:06<18:27,  2.54it/s]

pdbid: 8A2C	chain: A	error: 'UNK'
pdbid: 8A2C	chain: B	error: 'UNK'


 88%|████████▊ | 7723/8729 [31:25<07:37,  2.20it/s]

pdbid: 8EM8	chain: U	error: 'UNK'
pdbid: 8EM8	chain: U	error: 'UNK'
pdbid: 8EM8	chain: U	error: 'UNK'
pdbid: 8EM8	chain: U	error: 'UNK'


100%|██████████| 8729/8729 [39:36<00:00,  3.67it/s]


In [2]:
from utils.data_process import build_tokenizer_dataset, tokenizer_json_to_txt

pdb_paths = '/mnt/g/Little-LL/pretrain_pdb/'
output_path ='/mnt/g/Little-LL/pretrain_tokenizer/'

tokenizer_json_to_txt(output_path)